In [1]:
'''
%pip install finvizfinance
%pip install pandas
%pip install transformers
%pip install yfinance
%pip install goose3
%pip install requests
%pip install ipywidgets

%pip install torch
%pip install tensorflow
%pip install nltk
import nltk
nltk.download('punkt')
'''


In [ ]:
#from finvizfinance.screener.overview import Overview
#foverview2 = Overview()
#foverview2.get_filters()
#
#foverview2.get_filter_options('Shares Outstanding')

In [ ]:

from finvizfinance.screener.overview import Overview
import pandas as pd
import csv
import os

from transformers import pipeline
import yfinance as yf
from goose3 import Goose
from requests import get

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#FILTERS_DICT = {'Debt/Equity':'Under 1',                 # Positive Operating Margin
#                'PEG':'Low (<1)',                        # Debt-to-Equity ratio under 1
#                'Operating Margin':'Positive (>0%)',     # Low P/B (under 1)
#                'P/B':'Low (<1)',                        # Low P/E ratio (under 15)
#                'P/E':'Low (<15)',                       # Low PEG ratio (under 1)
#                'InsiderTransactions':'Positive (>0%)<'} # Positive Insider Transactions


FILTERS_DICT = {
    'Performance 2': 'Today +10%',     # Day increase 10%
    'Relative Volume': 'Over 5',       # High Relative Volume
    'Price': 'Under $20',              # Price under 20 USD
    'Shares Outstanding': 'Under 10M'  # Float under 10 million
}


def get_filtered_stocks():
    """
    Returns a list of tickers with:

    """
    
    foverview = Overview()
    foverview.set_filter(filters_dict=FILTERS_DICT)
    df_overview = foverview.screener_view()
    if not os.path.exists('out'): #ensures you have an 'out' folder ready
        os.makedirs('out')
    df_overview.to_csv('out/Overview.csv', index=False)
    
    tickers = df_overview['Ticker'].to_list()
    df_overview.head()
    return tickers

get_filtered_stocks()



['COHN',
 'DATS',
 'KOSS',
 'MCVT',
 'MI',
 'MULN',
 'MYSZ',
 'NRXP',
 'PLTN',
 'PSTV',
 'SBFM',
 'SGE',
 'XELA']

In [54]:

from finvizfinance.quote import finvizfinance
quote = finvizfinance('SGE')

df = quote.ticker_inside_trader()
from datetime import datetime
# Get today's date
today = datetime.today().date()

# Get the news and filter it to get only today's news
df = quote.ticker_news()
df = df[df['Date'].dt.date == today]
df


df = quote.ticker_fundament()
df

{'Company': 'Strong Global Entertainment Inc',
 'Sector': 'Technology',
 'Industry': 'Computer Hardware',
 'Country': 'USA',
 'Exchange': 'AMEX',
 'Index': '-',
 'P/E': '48.59',
 'EPS (ttm)': '0.05',
 'Insider Own': '78.01%',
 'Shs Outstand': '7.88M',
 'Perf Week': '100.00%',
 'Market Cap': '19.07M',
 'Forward P/E': '-',
 'EPS next Y': '-',
 'Insider Trans': '0.02%',
 'Shs Float': '1.73M',
 'Perf Month': '68.06%',
 'Income': '0.35M',
 'PEG': '-',
 'EPS next Q': '-',
 'Inst Own': '0.17%',
 'Short Float': '0.02%',
 'Perf Quarter': '108.62%',
 'Sales': '49.00M',
 'P/S': '0.39',
 'EPS this Y': '-',
 'Inst Trans': '-',
 'Short Ratio': '0.02',
 'Perf Half Y': '24.74%',
 'Book/sh': '0.99',
 'P/B': '2.45',
 'EPS next Y Percentage': '-',
 'ROA': '12.53%',
 'Short Interest': '0.00M',
 'Perf Year': '-',
 'Cash/sh': '0.69',
 'P/C': '3.49',
 'EPS next 5Y': '-',
 'ROE': '38.64%',
 '52W Range From': '1.02',
 '52W Range To': '4.35',
 'Perf YTD': '51.38%',
 'Dividend Est.': '-',
 'P/FCF': '14.56',
 'EP

In [67]:
def get_ticker_news_sentiment(ticker):
    """
    Returns a Pandas dataframe of the given ticker's most recent news article headlines,
    with the overal sentiment of each article.

    Args:
        ticker (string)

    Returns:
        pd.DataFrame: {'Date', 'Article title', Article sentiment'}
    """
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

    ticker_news = yf.Ticker(ticker)
    news_list = ticker_news.get_news()
    extractor = Goose()
    pipe = pipeline("text-classification", model="ProsusAI/finbert")
    data = []
    for dic in news_list:
        title = dic['title']
        response = get(dic['link'])
        article = extractor.extract(raw_html=response.content)
        text = article.cleaned_text
        date = article.publish_date
        if len(text) > 512:
        #if len(tokenizer.encode(text)) > 512:
            #inputs = tokenizer.encode_plus(
            #    text,
            #    max_length=510,
            #    truncation='longest_first',  # Truncate the longest sequences first
            #    padding='max_length',  # Pad sequences to the max length
            #    return_tensors='pt',  # Return PyTorch tensors
            #)
            #
            ## Convert tensor to list and then to string
            #input_ids = inputs["input_ids"].tolist()[0]
            #input_str = tokenizer.decode(input_ids)
            
                # Split the text into sentences
            from nltk.tokenize import sent_tokenize
            sentences = sent_tokenize(text)

            # Initialize an empty string for the new text
            new_text = ''

            # Add sentences to the new text until it exceeds 512 tokens
            for sentence in sentences:
                #tokens = tokenizer.encode(sentence, add_special_tokens=False)
                if len(new_text.split()) + len(sentence) > 512:
                #if len(tokenizer.encode(new_text + ' ' + sentence, add_special_tokens=False)) < 512:
                    new_text += ' ' + sentence
                break

            # Now you can pass 'inputs' to your model
            results = pipe(new_text)

            data.append({'Date':f'{date}',
                        'Article title':f'{title}',
                         'Article sentiment':results[0]['label']})

        else:
            results = pipe(text)
            data.append({'Date':f'{date}',
                         'Article title':f'{title}',
                         'Article sentiment':results[0]['label']})
    df = pd.DataFrame(data)
    return df

def generate_csv(ticker):
    get_ticker_news_sentiment(ticker).to_csv(f'out/{ticker}.csv', index=False)

undervalued = get_filtered_stocks()



sentiments = []
for ticker in undervalued:
#    generate_csv(ticker)
    sentiments.append(get_ticker_news_sentiment(ticker))



In [64]:
df

{'Company': 'Strong Global Entertainment Inc',
 'Sector': 'Technology',
 'Industry': 'Computer Hardware',
 'Country': 'USA',
 'Exchange': 'AMEX',
 'Index': '-',
 'P/E': '48.59',
 'EPS (ttm)': '0.05',
 'Insider Own': '78.01%',
 'Shs Outstand': '7.88M',
 'Perf Week': '100.00%',
 'Market Cap': '19.07M',
 'Forward P/E': '-',
 'EPS next Y': '-',
 'Insider Trans': '0.02%',
 'Shs Float': '1.73M',
 'Perf Month': '68.06%',
 'Income': '0.35M',
 'PEG': '-',
 'EPS next Q': '-',
 'Inst Own': '0.17%',
 'Short Float': '0.02%',
 'Perf Quarter': '108.62%',
 'Sales': '49.00M',
 'P/S': '0.39',
 'EPS this Y': '-',
 'Inst Trans': '-',
 'Short Ratio': '0.02',
 'Perf Half Y': '24.74%',
 'Book/sh': '0.99',
 'P/B': '2.45',
 'EPS next Y Percentage': '-',
 'ROA': '12.53%',
 'Short Interest': '0.00M',
 'Perf Year': '-',
 'Cash/sh': '0.69',
 'P/C': '3.49',
 'EPS next 5Y': '-',
 'ROE': '38.64%',
 '52W Range From': '1.02',
 '52W Range To': '4.35',
 'Perf YTD': '51.38%',
 'Dividend Est.': '-',
 'P/FCF': '14.56',
 'EP